# Defining the Fully Convolutional Neural Network

In [3]:
import os
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

In [2]:
class FCNN(nn.Module):
    
    def __init__(self):
        super(FCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=30, kernel_size=5, stride=2)
        
        self.conv2 = nn.Conv2d(in_channels=30, out_channels=60, kernel_size=3, stride=1)
        self.conv2_drop = nn.Dropout2d()
        
        self.conv3 = nn.Conv2d(in_channels=60, out_channels=60, kernel_size=3, stride=2)
        self.conv3_drop = nn.Dropout2d()
        
        self.conv4 = nn.Conv2d(in_channels=60, out_channels=120, kernel_size=3, stride=1)
        
        self.conv5 = nn.Conv2d(in_channels=120, out_channels=120, kernel_size=3, stride=1)
        
        self.conv5 = nn.Conv2d(in_channels=120, out_channels=1, kernel_size=3, stride=1)
        
        
    def forward(self, data):
        x = F.relu(self.conv1(data))
        x = F.relu(self.conv2_drop(self.conv2(x)))
        x = F.relu(self.conv3_drop(self.conv3(x)))
        x = F.relu(self.conv4(x))
        x = torch.sigmoid(self.conv5(x))
        return x
        
network = FCNN()

# Loading the samples

In [4]:
POLLEN_DATA_CREATED = False
cur_dir = os.getcwd()
pollen_files = cur_dir + '/Data/PollenData/'
np_pollen_data = []     # single date will be [img,label]


if not POLLEN_DATA_CREATED:
    for folder in next(os.walk(pollen_files))[1]:
        if folder == 'Real':
            label = 1
        else:
            label = 0
        parent_path = os.path.join(pollen_files, folder)
        for file in os.listdir(parent_path):
            if '.png' in file:
                try:
                    path = os.path.join(parent_path, file)
                    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
                    np_pollen_data.append([np.array(img), label])
                except Exception as e:
                    print(folder, file, str(e))

    np.random.shuffle(np_pollen_data)
    print("Data created.")


Data created.


In [5]:
# seperate the images and the labels into two lists:

images = []
labels = []

for i in range(len(np_pollen_data)):
    images.append(np_pollen_data[i][0])
    labels.append(np_pollen_data[i][1])

print(len(images))

3246


In [ ]:
#Create Dataloader and augment it:

dataloader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(torch.Tensor(images),torch.Tensor(label)),batch_size=10)


In [5]:
# Split into train and validation set:

train_x = images[:int(len(images)*0.8)]
train_y = labels[:int(len(images)*0.8)]
valid_x = images[int(len(images)*0.8):]
valid_y = labels[int(len(images)*0.8):]

In [6]:
print(images[1].shape)

(32, 32)


In [7]:
# Create Dataloaders:

trainloader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(torch.Tensor(train_x),torch.Tensor(train_y)),batch_size=10)
validloader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(torch.Tensor(valid_x),torch.Tensor(valid_y)),batch_size=100)

# Data Augmentation
and splitting the dataset into train and validation data. 

In [8]:
#transform to tensor
pollen_data = [[torch.from_numpy(i[0]).view(-1,32,32)/255.0,i[1]] for i in np_pollen_data]

#create datasets
data_amount = len(pollen_data)
train_size = int(data_amount*0.8)
val_size = int(data_amount*0.1)

pollen_training_set = pollen_data[:train_size]
pollen_validation_set = pollen_data[train_size:val_size]
pollen_testing_set = pollen_data[val_size:]

# Train the Network

First we have a look at the performance of the untrained model on our validation set. 

In [9]:
def validate(network, testloader):
    total = 0
    correct = 0
    # We do not need any gradiants here, since we do not train the network.
    # We are only interested in the predictions of the network on the testdata. 
    with torch.no_grad():
        
        for i, (inputs, labels) in enumerate(trainloader):
            outputs = network(torch.transpose(inputs[...,None],1,3)) 
            #print(labels.size(0))
            predicted = (outputs >= 0.9) # Predicted is a tensor of booleans 
            total += labels.size(0)
            predicted = predicted.view(predicted.size(0)) 
            #print(labels.size())
            #print((predicted == labels).sum().item())
            correct += (predicted == labels).sum().item()
    
    print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))
    
validate(network, validloader)

Accuracy of the network on the test images: 50 %


In [10]:
ep = 100;

#Which optimizer and criterion should we choose? Perform best?
criterion = nn.MSELoss()
optimizer = optim.SGD(network.parameters(), lr=0.001, momentum=0.9)


In [11]:
# from pytorch blitz tutorial

def train(network, trainloader, ep, criterion, optimizer, print_interval):
    for epoch in range(ep):
        
        running_loss = 0.0
    
        for i, (inputs, lables) in enumerate(trainloader):
    
            # zero the parameter gradients
            optimizer.zero_grad()
            #print(lables.shape)
            
            outputs = network(torch.transpose(inputs[...,None],1,3)).view(-1)
            #print(outputs.shape)
            loss = criterion(outputs, lables)
            loss.backward() #propagate the error back through the network
            optimizer.step() #adjust the weights of the network depending on the propagated error
    
            #that's it.
    
            #Some statistics:
            running_loss += loss.item()
    
            if i % print_interval == print_interval - 1:    # print every x mini-batches
                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / print_interval))
                running_loss = 0.0
    
    print('Finished Training')

train(network, trainloader, ep, criterion, optimizer, 32)

[1,    32] loss: 0.298
[1,    64] loss: 0.194
[1,    96] loss: 0.140
[1,   128] loss: 0.144
[1,   160] loss: 0.108
[1,   192] loss: 0.127
[1,   224] loss: 0.125
[1,   256] loss: 0.136
[2,    32] loss: 0.110
[2,    64] loss: 0.122
[2,    96] loss: 0.103
[2,   128] loss: 0.102
[2,   160] loss: 0.094
[2,   192] loss: 0.117
[2,   224] loss: 0.105
[2,   256] loss: 0.114
[3,    32] loss: 0.104
[3,    64] loss: 0.095
[3,    96] loss: 0.088
[3,   128] loss: 0.100
[3,   160] loss: 0.075
[3,   192] loss: 0.110
[3,   224] loss: 0.093
[3,   256] loss: 0.106
[4,    32] loss: 0.079
[4,    64] loss: 0.093
[4,    96] loss: 0.096
[4,   128] loss: 0.076
[4,   160] loss: 0.094
[4,   192] loss: 0.105
[4,   224] loss: 0.106
[4,   256] loss: 0.109
[5,    32] loss: 0.100
[5,    64] loss: 0.079
[5,    96] loss: 0.090
[5,   128] loss: 0.103
[5,   160] loss: 0.078
[5,   192] loss: 0.100
[5,   224] loss: 0.088
[5,   256] loss: 0.105
[6,    32] loss: 0.096
[6,    64] loss: 0.098
[6,    96] loss: 0.086
[6,   128] 

[44,    64] loss: 0.067
[44,    96] loss: 0.077
[44,   128] loss: 0.086
[44,   160] loss: 0.062
[44,   192] loss: 0.072
[44,   224] loss: 0.078
[44,   256] loss: 0.075
[45,    32] loss: 0.068
[45,    64] loss: 0.064
[45,    96] loss: 0.060
[45,   128] loss: 0.078
[45,   160] loss: 0.072
[45,   192] loss: 0.080
[45,   224] loss: 0.088
[45,   256] loss: 0.096
[46,    32] loss: 0.055
[46,    64] loss: 0.073
[46,    96] loss: 0.065
[46,   128] loss: 0.067
[46,   160] loss: 0.054
[46,   192] loss: 0.062
[46,   224] loss: 0.084
[46,   256] loss: 0.087
[47,    32] loss: 0.062
[47,    64] loss: 0.071
[47,    96] loss: 0.072
[47,   128] loss: 0.076
[47,   160] loss: 0.055
[47,   192] loss: 0.079
[47,   224] loss: 0.078
[47,   256] loss: 0.092
[48,    32] loss: 0.063
[48,    64] loss: 0.072
[48,    96] loss: 0.069
[48,   128] loss: 0.063
[48,   160] loss: 0.065
[48,   192] loss: 0.071
[48,   224] loss: 0.070
[48,   256] loss: 0.078
[49,    32] loss: 0.068
[49,    64] loss: 0.076
[49,    96] loss

[86,   256] loss: 0.090
[87,    32] loss: 0.045
[87,    64] loss: 0.059
[87,    96] loss: 0.062
[87,   128] loss: 0.066
[87,   160] loss: 0.049
[87,   192] loss: 0.071
[87,   224] loss: 0.067
[87,   256] loss: 0.073
[88,    32] loss: 0.052
[88,    64] loss: 0.055
[88,    96] loss: 0.063
[88,   128] loss: 0.058
[88,   160] loss: 0.051
[88,   192] loss: 0.060
[88,   224] loss: 0.071
[88,   256] loss: 0.080
[89,    32] loss: 0.051
[89,    64] loss: 0.054
[89,    96] loss: 0.065
[89,   128] loss: 0.061
[89,   160] loss: 0.051
[89,   192] loss: 0.064
[89,   224] loss: 0.071
[89,   256] loss: 0.071
[90,    32] loss: 0.057
[90,    64] loss: 0.062
[90,    96] loss: 0.075
[90,   128] loss: 0.054
[90,   160] loss: 0.053
[90,   192] loss: 0.069
[90,   224] loss: 0.068
[90,   256] loss: 0.079
[91,    32] loss: 0.040
[91,    64] loss: 0.062
[91,    96] loss: 0.059
[91,   128] loss: 0.058
[91,   160] loss: 0.048
[91,   192] loss: 0.062
[91,   224] loss: 0.067
[91,   256] loss: 0.080
[92,    32] loss

# Validation

The performance on the validation data after training:

In [12]:
validate(network, validloader)

Accuracy of the network on the test images: 91 %
